# Time Series Model Validation Demo

## 1. Introduction

## 2. Setup 

Prepare the environment for our analysis. First, **import** all necessary libraries and modules required for our analysis. Next, **connect** to the ValidMind MRM platform, which provides a comprehensive suite of tools and services for model validation.

Finally, define and **configure** the specific use case we are working on by setting up any required parameters, data sources, or other settings that will be used throughout the analysis. 

### Import Libraries

In [1]:
# Load API key and secret from environment variables
%load_ext dotenv
%dotenv .env

# ValidMind libraries 
import validmind as vm

### Connect to the ValidMind Library

In [2]:
vm.init(
  api_host = "http://localhost:3000/api/v1/tracking",
  project = "clhhz04x40000wcy6shay2oco"
)

Connected to ValidMind. Project: Customer Churn Model - Initial Validation (clhhz04x40000wcy6shay2oco)


### Find All Test Plans Available in the Developer Framework

In [3]:
vm.test_plans.list_plans()

ID,Name,Description
binary_classifier_metrics,BinaryClassifierMetrics,Test plan for sklearn classifier metrics
binary_classifier_validation,BinaryClassifierPerformance,Test plan for sklearn classifier models
binary_classifier_model_diagnosis,BinaryClassifierDiagnosis,Test plan for sklearn classifier model diagnosis tests
binary_classifier,BinaryClassifier,Test plan for sklearn classifier models that includes both metrics and validation tests
tabular_dataset,TabularDataset,Test plan for generic tabular datasets
tabular_dataset_description,TabularDatasetDescription,Test plan to extract metadata and descriptive statistics from a tabular dataset
tabular_data_quality,TabularDataQuality,Test plan for data quality on tabular datasets
normality_test_plan,NormalityTestPlan,Test plan to perform normality tests.
autocorrelation_test_plan,AutocorrelationTestPlan,Test plan to perform autocorrelation tests.
seasonality_test_plan,SesonalityTestPlan,Test plan to perform seasonality tests.


## 3. Load Models

#### Load Pre-Trained Time Series Models

In [4]:
# Currently only fred pre-trained models are available
from validmind.datasets.regression import fred as demo_dataset
model_A, train_df_A, test_df_A = demo_dataset.load_model('fred_loan_rates_model_1')
model_B, train_df_B, test_df_B = demo_dataset.load_model('fred_loan_rates_model_3')

#### Load Train Dataset

In [ ]:
train_df = demo_models.load_train_dataset()
train_df.head()

In [ ]:
train_df.tail()

#### Load Test Dataset

In [ ]:
test_df = demo_models.load_test_dataset()
test_df.head()

#### Create ValidMind Datasets

In [ ]:
vm_train_ds = vm.init_dataset(dataset=train_df, type="generic", target_column=demo_models.target_column)
vm_test_ds = vm.init_dataset(dataset=test_df, type="generic", target_column=demo_models.target_column)

In [ ]:
demo_models.target_column

## 4. Model Diagnosis

#### **Run Time Series Model Summary Test Plan**

## 5. Model Evaluation

### Performance across Models

#### **Run Time Series Model Performance Test Plan**

### Forecast vs Observed across Models 

#### **Run Time Series Forecast Test Plan**

In [ ]:
vm.test_plans.describe_plan("time_series_forecast")

In [ ]:
vm_model_A = vm.init_model(model = models['fred_loan_rates_model_3'],
                           test_ds=vm_test_ds)
vm_model_B = vm.init_model(model = models['fred_loan_rates_model_4'],
                           test_ds=vm_test_ds)
list_of_models = [vm_model_A, vm_model_B]

In [ ]:
config= {
    "model_prediction_ols": {
        "plot_start_date": '2010-01-01',
        "plot_end_date": '2022-01-01'
    }
}

vm.run_test_plan("time_series_forecast",
                                        models=list_of_models,
                                        config=config)